# Arabic Poetry Sentiment Analysis

In [1]:
#data load and preprocessing
import numpy as np
import pandas as pd 
import re 
import string

import nltk
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#ML Prepration
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score 

# Classification Model
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/muneraalrajhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/muneraalrajhi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/muneraalrajhi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Import and prepare the data

In [2]:
# reading the file
df = pd.read_excel("/Users/muneraalrajhi/Desktop/Capstone Project/CapstoneProject/data/Poems_Cleaned.xlsx")
df.head()

,Poem,Number of Versus,Poet,Century,Label,Type,Metre
0,أصابت سهام الحتف يا حسرة الدهرصريح قريش والخلا...,46,أبو المحاسن الكربلائي,العراق,عتاب,عموديه,بحر الطويل
1,جئت يا دهر بالرزية بكراغادرت مهجة الهدى وهي حر...,37,أبو المحاسن الكربلائي,العراق,عتاب,عموديه,بحر الخفيف
2,وشادن أورثني حبهكالائتلافيين حزنا طويلعز علي ا...,14,أبو المحاسن الكربلائي,العراق,عتاب,عموديه,بحر السريع
3,قد كانت الآمال معقودةفيك وللأمال تضليلواليوم ل...,11,أبو المحاسن الكربلائي,العراق,عتاب,عموديه,بحر السريع
4,سواي على الضيم يبقى رهيناوغيري يرى ضارعا مستكي...,87,أبو المحاسن الكربلائي,العراق,عتاب,عموديه,بحر المتقارب


In [3]:
# drop Unnecessary columns for the Sentiment Analysis
df = df.drop(columns=['Number of Versus','Poet','Century','Type','Metre']) 

In [4]:
df.head()

,Poem,Label
0,أصابت سهام الحتف يا حسرة الدهرصريح قريش والخلا...,عتاب
1,جئت يا دهر بالرزية بكراغادرت مهجة الهدى وهي حر...,عتاب
2,وشادن أورثني حبهكالائتلافيين حزنا طويلعز علي ا...,عتاب
3,قد كانت الآمال معقودةفيك وللأمال تضليلواليوم ل...,عتاب
4,سواي على الضيم يبقى رهيناوغيري يرى ضارعا مستكي...,عتاب


In [5]:
#data shape
df.shape

(12348, 2)

In [6]:
# check for null values
df.isna().sum()

Poem     0
Label    0
dtype: int64

In [7]:
# value count for each label
df['Label'].value_counts()

رثاء       2451
مدح        1454
حزن        1363
عتاب       1359
غزل        1358
شوق        1295
رومنسيه    1111
هجاء       1047
فراق        910
Name: Label, dtype: int64

## nlkt and text processing.

In [8]:
# define punctuations
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' 

# Expande Arabic StopWords
arabic_stopwords = set(nltk.corpus.stopwords.words("arabic"))
data = []
with open('/Users/muneraalrajhi/Desktop/Capstone Project/CapstoneProject/data/arabic_stop_words.txt','r') as myfile:
    for line in myfile:
        data.extend(map(str, line.rstrip('\n').split(',')))
arabic_stopwords.update(data)

#difine Arabic text stemmer
st = ISRIStemmer()

# 
def clean_text(text):
    text = text.translate(str.maketrans('','', punctuations))
    text = st.stem(text)
    text = ' '.join([word for word in text.split() if word not in arabic_stopwords])
    return text

In [9]:
df['Poem'] = df['Poem'].apply(clean_text)

## Bag of words

In [10]:
#Create 
max_features = 5000
count_vector = CountVectorizer(max_features = max_features)  
PoemBOW = count_vector.fit_transform(df['Poem']).toarray() 
PoemBOW

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
# BOW frequency
print(count_vector.vocabulary_)

{'سهام': 2706, 'الحتف': 587, 'حسرة': 2121, 'قريش': 3567, 'الدهر': 698, 'البرق': 480, 'غيث': 3312, 'الناس': 1315, 'قبلا': 3528, 'حملت': 2186, 'يعرف': 4904, 'حقيقة': 2145, 'شيء': 2822, 'أقوى': 220, 'خف': 2255, 'الشديد': 837, 'قلب': 3596, 'حكما': 2148, 'العدو': 972, 'غاب': 3248, 'أفق': 204, 'وجه': 4491, 'الصبح': 876, 'الوجه': 1386, 'السلامة': 798, 'الضحى': 902, 'فينا': 3505, 'حزنا': 2111, 'الليل': 1207, 'يحيى': 4790, 'آية': 13, 'الصبر': 877, 'بعده': 1601, 'التقى': 521, 'يتلو': 4761, 'النور': 1356, 'علينا': 3178, 'مجده': 3907, 'هاتيك': 4263, 'أجر': 54, 'جوهرا': 2031, 'زان': 2553, 'الوجود': 1387, 'جوهر': 2030, 'اللطف': 1194, 'الدنيا': 696, 'منار': 4060, 'يهتدي': 4986, 'ظلمة': 3015, 'يدرك': 4806, 'الفكر': 1100, 'الإسلام': 454, 'قيل': 3636, 'ساروا': 2594, 'القبر': 1113, 'يدا': 4802, 'بيضاء': 1680, 'الردى': 731, 'طي': 2997, 'المساعي': 1244, 'يزل': 4850, 'طيب': 2998, 'حياة': 2206, 'المرء': 1236, 'بالحمد': 1480, 'حولا': 2198, 'دين': 2367, 'نفسا': 4223, 'ماء': 3875, 'الباقي': 466, 'خير': 2286, 'م

In [12]:
# encode Label Column
encoder = LabelEncoder()
Label_encoder = encoder.fit_transform(df['Label'])
Label_encoder

array([4, 4, 4, ..., 5, 5, 5])

In [13]:
y = df['Label']

## Classification Model:

In [14]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(PoemBOW,Label_encoder, test_size =0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9878, 5000), (2470, 5000), (9878,), (2470,))

### LogisticRegression

In [15]:
logModel = LogisticRegression()
# train model
logModel.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = logModel.predict(X_test) 

# print the accuracy and classification_report
LogisticRegressionAcc = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.26      0.23      0.25       275
           1       0.32      0.41      0.36       479
           2       0.35      0.32      0.33       226
           3       0.43      0.40      0.42       253
           4       0.18      0.17      0.18       279
           5       0.32      0.34      0.33       260
           6       0.34      0.23      0.27       187
           7       0.57      0.54      0.55       293
           8       0.24      0.23      0.24       218

    accuracy                           0.33      2470
   macro avg       0.33      0.32      0.33      2470
weighted avg       0.34      0.33      0.33      2470



/Users/muneraalrajhi/opt/anaconda3/envs/tf/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Multinomial Naive Bayes

In [16]:
MNBModel = MultinomialNB()
# train model
MNBModel.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = MNBModel.predict(X_test) 

# print the accuracy and classification_report
MultinomialNBAcc = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.29      0.23      0.26       275
           1       0.38      0.35      0.37       479
           2       0.37      0.29      0.32       226
           3       0.35      0.45      0.40       253
           4       0.23      0.24      0.23       279
           5       0.35      0.45      0.39       260
           6       0.32      0.32      0.32       187
           7       0.57      0.64      0.61       293
           8       0.28      0.21      0.24       218

    accuracy                           0.36      2470
   macro avg       0.35      0.35      0.35      2470
weighted avg       0.35      0.36      0.35      2470



### RandomForestClassifier

In [17]:
RandomModel = RandomForestClassifier()
# train model
RandomModel.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = RandomModel.predict(X_test) 

# print the accuracy and classification_report
RandomForestClassifierAcc= accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.41      0.29      0.34       275
           1       0.33      0.48      0.39       479
           2       0.32      0.42      0.36       226
           3       0.52      0.49      0.50       253
           4       0.22      0.15      0.18       279
           5       0.30      0.27      0.29       260
           6       0.43      0.26      0.32       187
           7       0.49      0.59      0.54       293
           8       0.31      0.22      0.26       218

    accuracy                           0.37      2470
   macro avg       0.37      0.35      0.35      2470
weighted avg       0.37      0.37      0.36      2470



### LinearSVC

In [18]:
SVCModel = LinearSVC()

# train model
SVCModel.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = SVCModel.predict(X_test) 

# print the accuracy and classification_report
LinearSVCAcc= accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.23      0.23      0.23       275
           1       0.32      0.31      0.32       479
           2       0.30      0.30      0.30       226
           3       0.36      0.35      0.36       253
           4       0.19      0.20      0.19       279
           5       0.29      0.32      0.30       260
           6       0.25      0.22      0.23       187
           7       0.50      0.48      0.49       293
           8       0.23      0.25      0.24       218

    accuracy                           0.30      2470
   macro avg       0.30      0.30      0.30      2470
weighted avg       0.30      0.30      0.30      2470



/Users/muneraalrajhi/opt/anaconda3/envs/tf/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Best model:

In [19]:
models = pd.DataFrame({
                      'Model': ['Logistic Regression', 'Multinomial Naive Bayes', 'Random Forest', 'LinearSVC'],
                      'Score': [LogisticRegressionAcc, MultinomialNBAcc, RandomForestClassifierAcc, LinearSVCAcc]})
models.sort_values(by='Score', ascending=False)

,Model,Score
2,Random Forest,0.367611
1,Multinomial Naive Bayes,0.359109
0,Logistic Regression,0.333603
3,LinearSVC,0.300810


## Testing:

In [20]:
#qoute
test_text = [' لعمري لقد أوهيت قلبي عن العزا وطأطأت رأسي والفؤاد كئيب لقد قصمت مني قناة صليبة ويقصم عود النبع وهو صليب']
# حزن
# encoding
test_vector = count_vector.transform(test_text)
test_vector = test_vector.toarray()

## Perform and Evaluate the Model 
text_predict_class = encoder.inverse_transform(RandomModel.predict(test_vector))
print(test_text)
print(text_predict_class)

[' لعمري لقد أوهيت قلبي عن العزا وطأطأت رأسي والفؤاد كئيب لقد قصمت مني قناة صليبة ويقصم عود النبع وهو صليب']
['رومنسيه']


In [21]:
#qoute
test_text = ['إن وترت قلبك الهموم فما مثل انتصار بالناي والوتر وشادن حيرت لواحظه ألحاظ عين الغزال بالحور'] 
# غزل
# encoding
test_vector = count_vector.transform(test_text)
test_vector = test_vector.toarray()

## Perform and Evaluate the Model 
text_predict_class = encoder.inverse_transform(RandomModel.predict(test_vector))
print(test_text)
print(text_predict_class)

['إن وترت قلبك الهموم فما مثل انتصار بالناي والوتر وشادن حيرت لواحظه ألحاظ عين الغزال بالحور']
['غزل']


#### References:
- Count Vectorizer vs TFIDF Vectorizer | Natural Language Processing: https://www.linkedin.com/pulse/count-vectorizers-vs-tfidf-natural-language-processing-sheel-saket/
- Product-Categorization-NLP: https://github.com/aniass/Product-Categorization-NLP/blob/master/Text_analysis.ipynb